#  Instrution on how to run the code

In this code, the whole jupyter notebook could be segmented into three majot part, training, predicting and model resuming. 

The previous four cells are basic configuration, including mount the google drive, download and unzip the kaggle dataset. The forth cell is to check the type of GPU.

The first main section is training section. I referred and modified the argument dictionary to allow easy hyperparameter tuning. You could modify the regarding hyperparameter to control the tuning process. 

The second main code chunk is the model prediction. I would load the saved model to make prediction and submit to the kaggle. You have to maintain the hyperparameter same with loaded model.

The third main code chunk is to resume the model training. Therefore, apart from the same hyperparameters in previous two code. I added another `resume_epoch`, which would indicate the last epoch in the resuming model training part and  `epoch` would be the start epoch of the model.

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
%cd /content
!mkdir drive
%cd drive
%cd ..
!google-drive-ocamlfuse /content/drive

In [ ]:
import json

TOKEN = {"username":"yuxuanwucmu","key":"81ded4babfd327efdc7655be369299b5"}

! pip install kaggle==1.5.12
! mkdir -p .kaggle
! mkdir -p /content & mkdir -p /content/.kaggle & mkdir -p /root/.kaggle/

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(TOKEN, file)

! pip install --upgrade --force-reinstall --no-deps kaggle
! ls "/content/.kaggle"
! chmod 600 /content/.kaggle/kaggle.json
! cp /content/.kaggle/kaggle.json /root/.kaggle/

! kaggle config set -n path -v /content/drive

In [ ]:
! kaggle competitions download -c 11-785-s22-hw1p2
! unzip /content/drive/competitions/11-785-s22-hw1p2/11-785-s22-hw1p2.zip

In [ ]:
!nvidia-smi

Fri Feb 11 12:29:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.metrics import accuracy_score
from collections import Counter
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.utils.data import ConcatDataset

import time
import sys

class Layer(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.25):
        super(Layer, self).__init__()

        if args['activation'].lower() == "Tanh".lower():
            nn_activation = nn.Tanh()
        elif args['activation'].lower() == "LeakyReLU".lower():
            nn_activation = nn.LeakyReLU()
        elif args['activation'].lower() == "ELU".lower():
            nn_activation = nn.ELU()
        else:
            nn_activation = nn.ReLU()

        standard_layer = [
            nn.Linear(in_features=in_size, out_features=out_size),
            nn.BatchNorm1d(out_size),
            nn_activation,
            nn.Dropout(dropout)
        ]
        self.layer = nn.Sequential(*standard_layer)

    def forward(self, x):
        return self.layer(x)
        
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # each is 13 dim letter
        context = args['context']
        in_size = 13 * (2 * context + 1)
        layers = [
           Layer(in_size, 4096),
            Layer(4096, 4096),
            Layer(4096, 4096),
            Layer(4096, 4096),
            Layer(4096, 4096),
            Layer(4096, 2048),            
            Layer(2048, 1024),
            Layer(1024, 40),
        ]
        self.laysers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.laysers(A0)
        return x


class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, shuffle=True, partition="dev-clean", csvpath=None, submission=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample

        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.Y_dir = data_path + "/" + partition + "/transcript/"

        self.X_names = os.listdir(self.X_dir)
        self.Y_names = os.listdir(self.Y_dir)

        # using a small part of the dataset to debug
        if csvpath:
            subset = self.parse_csv(csvpath)
            self.X_names = [i for i in self.X_names if i in subset]
            self.Y_names = [i for i in self.Y_names if i in subset]

        if shuffle == True:
            XY_names = list(zip(self.X_names, self.Y_names))
            random.shuffle(XY_names)
            self.X_names, self.Y_names = zip(*XY_names)

        assert (len(self.X_names) == len(self.Y_names))
        self.length = len(self.X_names)

        self.PHONEMES = [
            'SIL', 'AA', 'AE', 'AH', 'AO', 'AW', 'AY',
            'B', 'CH', 'D', 'DH', 'EH', 'ER', 'EY',
            'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K',
            'L', 'M', 'N', 'NG', 'OW', 'OY', 'P',
            'R', 'S', 'SH', 'T', 'TH', 'UH', 'UW',
            'V', 'W', 'Y', 'Z', 'ZH', '<sos>', '<eos>']

    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[1])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))

    def __getitem__(self, i):
        sample_range = range(i * self.sample, min((i + 1) * self.sample, self.length))

        X, Y = [], []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
            Y_path = self.Y_dir + self.Y_names[j]

            label = [self.PHONEMES.index(yy) for yy in np.load(Y_path)][1:-1]

            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0)) / X_data.std(axis=0)
            X.append(X_data)
            Y.append(np.array(label))

        X, Y = np.concatenate(X), np.concatenate(Y)
        return X, Y



class LibriItems(torch.utils.data.Dataset):
    def __init__(self, X, Y, context=0):
        assert (X.shape[0] == Y.shape[0])

        self.length = X.shape[0]
        self.context = context

        if context == 0:
            self.X, self.Y = X, Y
        else:
            self.X = np.pad(X, ((context, context), (0, 0)), 'constant', constant_values=0)
            self.Y = Y
            pass

    def __len__(self):
        return self.length

    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
            yy = self.Y[i]
        else:
            start_i = i
            end_i = i + 2 * self.context + 1
            xx = self.X[start_i:end_i].flatten()
            yy = self.Y[i]

            pass
        return xx, yy


def train(args, model, device, train_samples, optimizer, criterion, epoch, scaler):
    model.train()
    running_loss = 0.0      
    for i in range(len(train_samples)):
        X, Y = train_samples[i]
        train_items = LibriItems(X, Y, context=args['context'])
        train_loader = torch.utils.data.DataLoader(train_items, batch_size=args['batch_size'], shuffle=True)

        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.float().to(device)
            target = target.long().to(device)

            with torch.cuda.amp.autocast():
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)

                # loss.backward()

                scaler.scale(loss).backward()  # get gradients w.r.t the loss
                scaler.step(optimizer)  # update the weights

                scaler.update()
                if batch_idx % args['log_interval'] == 0:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(train_loader.dataset),
                               100. * batch_idx / len(train_loader), loss.item()))

    running_loss = running_loss / len(train_samples)
    # print("Training loss is: ",running_loss)
    return running_loss


def test(args, model, device, dev_samples, criterion):
    model.eval()
    running_loss = 0.0    
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
        for i in range(len(dev_samples)):
            X, Y = dev_samples[i]

            test_items = LibriItems(X, Y, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

            for data, true_y in test_loader:
                data = data.float().to(device)
                true_y = true_y.long().to(device)

                output = model(data)
                pred_y = torch.argmax(output, axis=1)
                loss = criterion(output,true_y)
                running_loss+=loss.item()

                pred_y_list.extend(pred_y.tolist())
                true_y_list.extend(true_y.tolist())
    
    running_loss = running_loss / len(dev_samples)
    # print("Dev loss is: ",running_loss)
    train_accuracy = accuracy_score(true_y_list, pred_y_list)
    return train_accuracy, running_loss


def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)


def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = Network().to(device)
    optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    criterion = torch.nn.CrossEntropyLoss()
    # scheduler1 = StepLR(optimizer, step_size=4, gamma=0.1)
    scheduler1 = ReduceLROnPlateau(optimizer, factor=0.5,patience=2, mode='max',threshold=0.01,verbose=True)

    scaler = torch.cuda.amp.GradScaler()


    # If you want to use full Dataset, please pass None to csvpath
    train_samples = LibriSamples(data_path=args['LIBRI_PATH'], shuffle=True, partition="train-clean-100",
                                 csvpath= None,submission=args['submission'])
    dev_samples = LibriSamples(data_path=args['LIBRI_PATH'], shuffle=True, partition="dev-clean")
    if args['submission']:
        train_samples = ConcatDataset([train_samples,dev_samples])

    create_folder("/content/drive/hw1/checkpoints")

    save_path_dir = args['model_type']+ "_context_"+str(args['context']) + "_lr_"+str(args['lr'])+"_"+args['schedular']+"_"+args['activation']
    model_path= "/content/drive/hw1/checkpoints/"+save_path_dir+"/"
    args['save_path_dir'] = model_path
    create_folder(model_path)
    print("We are going to save to "+ model_path)
    
    txt_file = args['save_path_dir']+"dev_acc.txt"
    with open(txt_file,'w') as f :

        for epoch in range(1, args['epoch'] + 1):
            start = time.time()
            # train the model, with parameters changed in the train step
            train(args, model, device, train_samples, optimizer, criterion, epoch, scaler )

            # validation step, with validation accuracy generated
            test_acc, dev_loss = test(args, model, device, dev_samples,criterion)
            scheduler1.step(test_acc)

            print('Dev accuracy: ', test_acc)
            print_content = "Dev accuracy in epoch {} is {}\n".format(epoch,test_acc)
            f.write(print_content)

            end = time.time()
            print("", (end - start)/60,"minutes")

            # specify the path
            path = "/content/drive/hw1/checkpoints/{}/model_epoch_{}.txt".format(save_path_dir,epoch)

            # save the checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(), 
                'scheduler': scheduler1,
            }, path)


if __name__ == '__main__':
    args = {
        'batch_size': 2048,
        'context': 30,
        'log_interval': 200,
        'LIBRI_PATH': '/content/hw1p2_student_data',
        'lr': 1e-3,
        'epoch': 30,
        'save_path_dir':'',
        'model_type':'Pyramid',
        'schedular':'ReduceLROnPlateau_factor0.5_max_patience_2',
        'activation':'LeakyReLU',
        'submission':False,
    }
    main(args)

We are going to save to /content/drive/hw1/checkpoints/Pyramid_context_30_lr_0.001_ReduceLROnPlateau_factor0.5_max_patience_2_submission_LeakyReLU/
Train Epoch: 1 [0/25340216 (0%)]	Loss: 4.093301
Train Epoch: 1 [409600/25340216 (2%)]	Loss: 2.097481
Train Epoch: 1 [819200/25340216 (3%)]	Loss: 2.023282
Train Epoch: 1 [1228800/25340216 (5%)]	Loss: 1.911462
Train Epoch: 1 [1638400/25340216 (6%)]	Loss: 1.840209
Train Epoch: 1 [2048000/25340216 (8%)]	Loss: 1.788753
Train Epoch: 1 [2457600/25340216 (10%)]	Loss: 1.773855
Train Epoch: 1 [2867200/25340216 (11%)]	Loss: 1.820575
Train Epoch: 1 [3276800/25340216 (13%)]	Loss: 1.735468
Train Epoch: 1 [3686400/25340216 (15%)]	Loss: 1.685440
Train Epoch: 1 [4096000/25340216 (16%)]	Loss: 1.677622
Train Epoch: 1 [4505600/25340216 (18%)]	Loss: 1.715910
Train Epoch: 1 [4915200/25340216 (19%)]	Loss: 1.711706
Train Epoch: 1 [5324800/25340216 (21%)]	Loss: 1.641915
Train Epoch: 1 [5734400/25340216 (23%)]	Loss: 1.649227
Train Epoch: 1 [6144000/25340216 (24%)]	L

KeyboardInterrupt: ignored

In [ ]:
# Model prediction

In [ ]:
from numpy.lib.npyio import save
import os
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import StepLR,ReduceLROnPlateau

class Layer(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.25):
        super(Layer, self).__init__()

        if args['activation'].lower() == "Tanh".lower():
            nn_activation = nn.Tanh()
        elif args['activation'].lower() == "LeakyReLU".lower():
            nn_activation = nn.LeakyReLU()
        elif args['activation'].lower() == "ELU".lower():
            nn_activation = nn.ELU()
        else:
            nn_activation = nn.ReLU()

        standard_layer = [
            nn.Linear(in_features=in_size, out_features=out_size),
            nn.BatchNorm1d(out_size),
            nn_activation,
            nn.Dropout(dropout)
        ]
        self.layer = nn.Sequential(*standard_layer)

    def forward(self, x):
        return self.layer(x)
        
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # each is 13 dim letter
        context = args['context']
        in_size = 13 * (2 * context + 1)
        layers = [
           Layer(in_size, 4096),
            Layer(4096, 4096),
            Layer(4096, 4096),
            Layer(4096, 4096),
            Layer(4096, 4096),
            Layer(4096, 2048),            
            Layer(2048, 1024),
            Layer(1024, 40),
        ]
        self.laysers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.laysers(A0)
        return x


class LibriSamplesTest(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, partition="test-clean", csvpath=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample

        self.X_dir = data_path + "/" + partition + "/mfcc/"

        # using a small part of the dataset to debug
        if csvpath:
            subset = self.parse_csv(csvpath)
            # self.X_names = [i for i in self.X_names if i in subset]
            self.X_names = [i for i in subset]

        self.length = len(self.X_names)

    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[0])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))

    def __getitem__(self, i):
        sample_range = range(i * self.sample, min((i + 1) * self.sample, self.length))

        X = []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0)) / X_data.std(axis=0)
            X.append(X_data)

        X = np.concatenate(X)
        return X


class LibriItemsTest(torch.utils.data.Dataset):
    def __init__(self, X, context=0):

        self.length = X.shape[0]
        self.context = context

        if context == 0:
            self.X = X
        else:
            self.X = np.pad(X, ((context, context), (0, 0)), 'constant', constant_values=0)
            pass

    def __len__(self):
        return self.length

    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
        else:
            start_i = i
            end_i = i + 2 * self.context + 1
            xx = self.X[start_i:end_i].flatten()
            pass
        return xx


def predict(args, model, device, test_samples):
    model.eval()
    res = []
    with torch.no_grad():
        for i in range(len(test_samples)):
            X = test_samples[i]

            test_items = LibriItemsTest(X, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

            for data in test_loader:
                data = data.float().to(device)

                output = model(data)
                pred_y = torch.argmax(output, axis=1)
                res.extend(pred_y.tolist())
    return res


def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

args = {
    'batch_size': 2048,
    'context': 30,
    'log_interval': 200,
    'LIBRI_PATH': '/content/hw1p2_student_data',
    'lr': 1e-3,
    'epoch': 100,
    'save_path_dir':'',
    'model_type':'Pyramid',
    'schedular':'ReduceLROnPlateau_factor0.5_max_patience_2',
    'activation':'LeakyReLU',
    'submission':False,
}
save_path_dir = args['model_type']+ "_context_"+str(args['context']) + "_lr_"+str(args['lr'])+"_"+args['schedular']+"_"+args['activation']
# save_path_dir = args['model_type']+ "_context_"+str(args['context']) + "_lr_"+str(args['lr'])+"_"+args['schedular']

print("We are predicting files from",save_path_dir)

model_path= "/content/drive/hw1/checkpoints/"+save_path_dir+"/"
args['save_path_dir'] = model_path

results_path = "/content/drive/hw1/results/{}".format(save_path_dir)
create_folder(results_path)

# Reload the model and necessary architecture
model = Network().to(device)
optimizer = optim.Adam(model.parameters(), lr=args['lr'])
criterion = torch.nn.CrossEntropyLoss()
# scheduler1 = StepLR(optimizer, step_size=4, gamma=0.1)
scheduler1 = ReduceLROnPlateau(optimizer, factor=0.6,patience=2, mode='max',threshold=0.01,verbose=True)

# Reload the checkpoint
checkpoint = torch.load("/content/drive/hw1/checkpoints/{}/model_epoch_{}.txt".format(save_path_dir,args['epoch']))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler1=checkpoint['scheduler']
epoch = checkpoint['epoch']

test_samples = LibriSamplesTest(data_path=args['LIBRI_PATH'], partition="test-clean",
                                csvpath="/content/test_order.csv")

pred_Y = predict(args, model, device, test_samples)
npy_path = '/content/drive/hw1/results/{}/predict_labels_{}.npy'.format(save_path_dir,args['epoch'])
np.save(npy_path, pred_Y)
label = np.load(npy_path)

id = np.array(range(len(label)))
df = pd.DataFrame({"id": id, "label": label})
create_folder("/content/drive/hw1/results/kaggle/")
df.to_csv("/content/drive/hw1/results/kaggle/submission.csv", index=False)
! kaggle competitions submit -c 11-785-s22-hw1p2 -f /content/drive/hw1/results/kaggle/submission.csv -m "Message"

We are predicting files from Pyramid_context_30_lr_0.001_ReduceLROnPlateau_factor0.5_max_patience_2_LeakyReLU
100% 18.6M/18.6M [00:03<00:00, 4.99MB/s]
Successfully submitted to Frame-Level Speech Recognition

In [ ]:
!nvidia-smi

Wed Feb  9 19:04:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.metrics import accuracy_score
from collections import Counter
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.utils.data import ConcatDataset

import time
import sys

class Layer(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.25):
        super(Layer, self).__init__()

        if args['activation'].lower() == "Tanh".lower():
            nn_activation = nn.Tanh()
        elif args['activation'].lower() == "LeakyReLU".lower():
            nn_activation = nn.LeakyReLU()
        elif args['activation'].lower() == "ELU".lower():
            nn_activation = nn.ELU()
        else:
            nn_activation = nn.ReLU()

        standard_layer = [
            nn.Linear(in_features=in_size, out_features=out_size),
            nn.BatchNorm1d(out_size),
            nn_activation,
            nn.Dropout(dropout)
        ]
        self.layer = nn.Sequential(*standard_layer)

    def forward(self, x):
        return self.layer(x)
        
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # each is 13 dim letter
        context = args['context']
        in_size = 13 * (2 * context + 1)
        layers = [
           Layer(in_size, 4096),
            Layer(4096, 4096),
            Layer(4096, 4096),
            Layer(4096, 4096),
            Layer(4096, 4096),
            Layer(4096, 2048),            
            Layer(2048, 1024),
            Layer(1024, 40),
        ]
        self.laysers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.laysers(A0)
        return x


class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, shuffle=True, partition="dev-clean", csvpath=None, submission=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample

        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.Y_dir = data_path + "/" + partition + "/transcript/"

        self.X_names = os.listdir(self.X_dir)
        self.Y_names = os.listdir(self.Y_dir)

        # using a small part of the dataset to debug
        if csvpath:
            subset = self.parse_csv(csvpath)
            self.X_names = [i for i in self.X_names if i in subset]
            self.Y_names = [i for i in self.Y_names if i in subset]

        if shuffle == True:
            XY_names = list(zip(self.X_names, self.Y_names))
            random.shuffle(XY_names)
            self.X_names, self.Y_names = zip(*XY_names)

        assert (len(self.X_names) == len(self.Y_names))
        self.length = len(self.X_names)

        self.PHONEMES = [
            'SIL', 'AA', 'AE', 'AH', 'AO', 'AW', 'AY',
            'B', 'CH', 'D', 'DH', 'EH', 'ER', 'EY',
            'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K',
            'L', 'M', 'N', 'NG', 'OW', 'OY', 'P',
            'R', 'S', 'SH', 'T', 'TH', 'UH', 'UW',
            'V', 'W', 'Y', 'Z', 'ZH', '<sos>', '<eos>']

    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[1])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))

    def __getitem__(self, i):
        sample_range = range(i * self.sample, min((i + 1) * self.sample, self.length))

        X, Y = [], []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
            Y_path = self.Y_dir + self.Y_names[j]

            label = [self.PHONEMES.index(yy) for yy in np.load(Y_path)][1:-1]

            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0)) / X_data.std(axis=0)
            X.append(X_data)
            Y.append(np.array(label))

        X, Y = np.concatenate(X), np.concatenate(Y)
        return X, Y



class LibriItems(torch.utils.data.Dataset):
    def __init__(self, X, Y, context=0):
        assert (X.shape[0] == Y.shape[0])

        self.length = X.shape[0]
        self.context = context

        if context == 0:
            self.X, self.Y = X, Y
        else:
            self.X = np.pad(X, ((context, context), (0, 0)), 'constant', constant_values=0)
            self.Y = Y
            pass

    def __len__(self):
        return self.length

    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
            yy = self.Y[i]
        else:
            start_i = i
            end_i = i + 2 * self.context + 1
            xx = self.X[start_i:end_i].flatten()
            yy = self.Y[i]

            pass
        return xx, yy


def train(args, model, device, train_samples, optimizer, criterion, epoch, scaler):
    model.train()
    running_loss = 0.0      
    for i in range(len(train_samples)):
        X, Y = train_samples[i]
        train_items = LibriItems(X, Y, context=args['context'])
        train_loader = torch.utils.data.DataLoader(train_items, batch_size=args['batch_size'], shuffle=True)

        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.float().to(device)
            target = target.long().to(device)

            with torch.cuda.amp.autocast():
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)

                # loss.backward()

                scaler.scale(loss).backward()  # get gradients w.r.t the loss
                scaler.step(optimizer)  # update the weights

                scaler.update()
                if batch_idx % args['log_interval'] == 0:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(train_loader.dataset),
                               100. * batch_idx / len(train_loader), loss.item()))

    running_loss = running_loss / len(train_samples)
    # print("Training loss is: ",running_loss)
    return running_loss


def test(args, model, device, dev_samples, criterion):
    model.eval()
    running_loss = 0.0    
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
        for i in range(len(dev_samples)):
            X, Y = dev_samples[i]

            test_items = LibriItems(X, Y, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

            for data, true_y in test_loader:
                data = data.float().to(device)
                true_y = true_y.long().to(device)

                output = model(data)
                pred_y = torch.argmax(output, axis=1)
                loss = criterion(output,true_y)
                running_loss+=loss.item()

                pred_y_list.extend(pred_y.tolist())
                true_y_list.extend(true_y.tolist())
    
    running_loss = running_loss / len(dev_samples)
    # print("Dev loss is: ",running_loss)
    train_accuracy = accuracy_score(true_y_list, pred_y_list)
    return train_accuracy, running_loss


def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)


def main(args):
    save_path_dir = args['model_type']+ "_context_"+str(args['context']) + "_lr_"+str(args['lr'])+"_"+args['schedular']+"_"+args['activation']
    # save_path_dir = args['model_type']+ "_context_"+str(args['context']) + "_lr_"+str(args['lr'])+"_"+args['schedular']

    print("We are reloading files from",save_path_dir)

    model_path= "/content/drive/hw1/checkpoints/"+save_path_dir+"/"
    args['save_path_dir'] = model_path

    results_path = "/content/drive/hw1/results/{}".format(save_path_dir)
    # create_folder(results_path)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = Network().to(device)
    optimizer = optim.Adam(model.parameters(), lr=args['lr'])
    criterion = torch.nn.CrossEntropyLoss()
    # scheduler1 = StepLR(optimizer, step_size=4, gamma=0.1)
    scheduler1 = ReduceLROnPlateau(optimizer, factor=0.5,patience=2, mode='max',threshold=0.01,verbose=True)

    scaler = torch.cuda.amp.GradScaler()

    # Reload the checkpoint
    checkpoint = torch.load("/content/drive/hw1/checkpoints/{}/model_epoch_{}.txt".format(save_path_dir,args['resume_epoch']))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler1=checkpoint['scheduler']
    epoch = checkpoint['epoch']

    # If you want to use full Dataset, please pass None to csvpath
    train_samples = LibriSamples(data_path=args['LIBRI_PATH'], shuffle=True, partition="train-clean-100",
                                 csvpath= None,submission=args['submission'])
    dev_samples = LibriSamples(data_path=args['LIBRI_PATH'], shuffle=True, partition="dev-clean")
    
    if args['submission']:
        train_samples = ConcatDataset([train_samples,dev_samples])
    
    txt_file = args['save_path_dir']+'resumed_from_epoch'+str(args['resume_epoch'])+"dev_acc.txt"
    with open(txt_file,'w') as f :

        for epoch in range(args['resume_epoch']+1, args['epoch'] + 1):
            start = time.time()
            # train the model, with parameters changed in the train step
            train(args, model, device, train_samples, optimizer, criterion, epoch, scaler )

            # validation step, with validation accuracy generated
            test_acc, dev_loss = test(args, model, device, dev_samples,criterion)
            scheduler1.step(test_acc)

            print('Dev accuracy: ', test_acc)
            print_content = "Dev accuracy in epoch {} is {}\n".format(epoch,test_acc)
            f.write(print_content)

            end = time.time()
            print("", (end - start)/60,"minutes")

            # specify the path
            path = "/content/drive/hw1/checkpoints/{}/model_epoch_{}.txt".format(save_path_dir,epoch)

            # save the checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(), 
                'scheduler': scheduler1,
            }, path)

# /content/drive/hw1/checkpoints/Diamond_8_context_30_lr_0.001_ReduceLROnPlateau_factor0.6_max_LeakyReLU
if __name__ == '__main__':
    args = {
        'batch_size': 2048,
        'context': 30,
        'log_interval': 200,
        'LIBRI_PATH': '/content/hw1p2_student_data',
        'lr': 1e-3,
        'epoch': 120,
        'save_path_dir':'',
        'model_type':'Pyramid',
        'schedular':'ReduceLROnPlateau_factor0.5_max_patience_2',
        'activation':'LeakyReLU',
        'submission':False,
        'resume_epoch':100,
    }
    main(args)

We are reloading files from Pyramid_context_30_lr_0.001_ReduceLROnPlateau_factor0.5_max_patience_2_LeakyReLU
Train Epoch: 101 [0/25334306 (0%)]	Loss: 1.024348
Train Epoch: 101 [409600/25334306 (2%)]	Loss: 1.006572
Train Epoch: 101 [819200/25334306 (3%)]	Loss: 1.027281
Train Epoch: 101 [1228800/25334306 (5%)]	Loss: 1.005733
Train Epoch: 101 [1638400/25334306 (6%)]	Loss: 0.972232
Train Epoch: 101 [2048000/25334306 (8%)]	Loss: 0.988977
Train Epoch: 101 [2457600/25334306 (10%)]	Loss: 1.029442
Train Epoch: 101 [2867200/25334306 (11%)]	Loss: 1.025268
Train Epoch: 101 [3276800/25334306 (13%)]	Loss: 1.090567
Train Epoch: 101 [3686400/25334306 (15%)]	Loss: 1.011129
Train Epoch: 101 [4096000/25334306 (16%)]	Loss: 0.991853
Train Epoch: 101 [4505600/25334306 (18%)]	Loss: 1.017848
Train Epoch: 101 [4915200/25334306 (19%)]	Loss: 1.032083
Train Epoch: 101 [5324800/25334306 (21%)]	Loss: 1.064490
Train Epoch: 101 [5734400/25334306 (23%)]	Loss: 1.018716
Train Epoch: 101 [6144000/25334306 (24%)]	Loss: 1.